#目的
Pythonでスポーツ庁の調査を楽に集計したい

#手順
#1. 準備と個人情報の削除済みデータの読み込み
#2. MAデータの整理
#3. SA・MA・FAのデータを集計・出力+個別処理用データ出力
#4. 個別処理用データの集計・出力

#注意事項
本番で使うなら、高井課長の許可は要ると思ったが、VScodeとPythonのインストールで何とかなるかも…
絶対バグというか、浅はかな知識が及ばない箇所がある



# 実行

In [39]:
# 1.準備と個人情報の削除済みデータの読み込み
## 1.1.ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 1.2.データの読み込み
file_name = '/content/drive/MyDrive/【R5】ドライブでの加工テスト用.xlsx' #ファイル名に合わせて変更（ファイル名もしくはExcelファイルのパス（保存場所）

try: #下記のことをやってくれ！という命令
  # Excelファイルを読み込む
  personal_information_deleted_df = pd.read_excel(file_name)

  # データフレームを表示
  print(personal_information_deleted_df)
##1.3失敗したら教えてくれ！という命令
except FileNotFoundError:
  print(f"エラー: ファイル名 '{file_name}' が見つかりません.")
except Exception as e:
  print(f"An error occurred: {e}")

    num  Q1_3_1_1：休日の学校運動部活動の段階的な地域移行について  Q1_3_1_1：その他内容  \
0     1                                 2             NaN   
1     2                                 2             NaN   
2     3                                 3             NaN   
3     4                                 2             NaN   
4     5                                 1             NaN   
..  ...                               ...             ...   
87   88                                 1             NaN   
88   89                                 2             NaN   
89   90                                 2             NaN   
90   91                                 2             NaN   
91   92                                 3             NaN   

    Q1_3_1_R3_2:休日の学校運動部活動の段階的な地域移行の実施主体を担うことについて  \
0                                               4   
1                                               5   
2                                               3   
3                                               4   
4 

In [40]:
# 2.MAデータのスプリット（NAに未対応かも…）
## 2.1.スプリットをする関数
def split_multiple_columns(df, split_columns):
    # MAデータとしてスプリットしたことを覚えておいてもらうため準備
    column_mapping = {}

    for col_name, delimiter in split_columns.items():
        # Check if the column exists in the DataFrame before proceeding
        if col_name not in df.columns:
            print(f"Warning: Column '{col_name}' not found in DataFrame. Skipping.")
            continue  # Skip to the next column

        # 複数回答データをdelimiterで分割して、１つのリストにまとめるおまじない
        split_ans = [df[col_name][i].split(delimiter) for i in range(len(df))]
        # リストのflattening
        split_ans_ = []
        for s in split_ans:
            split_ans_.extend(s)
        # 重複を除きユニークな回答選択肢を取得
        split_ans_ = np.unique(split_ans_)
        # ダミー変数を作成
        split_dummy = []
        for j in range(len(split_ans_)):
            split_dummy.append([split_ans_[j] in df[col_name][i] for i in range(df.shape[0])])
        split_dummy = pd.DataFrame(split_dummy, index=split_ans_).T
        # 列名を変更
        split_dummy.columns = col_name + "_" + split_dummy.columns
        # 元のデータに結合
        df = pd.concat([df, split_dummy*1], axis=1)
        # MAデータとしてスプリットしたことを覚えておいてもらうためのおまじない
        column_mapping[col_name] = split_dummy.columns.tolist()

    return df, column_mapping
## 2.2MAの回答で分割したい列名を指定
split_columns = {
    'Q3_トッピング': '.', # カンマで区切っている場合は','に置き換え
    # 他のカラムも同様に追加
}
## 2.3関数を実行
personal_information_deleted_df,column_mapping = split_multiple_columns(personal_information_deleted_df, split_columns)
## 2.4実行結果を表示するおまじない
print(personal_information_deleted_df)
## 2.5結果をエクセルで出力してくれるおまじない（出力ファイル名を指定する必要がある）
personal_information_deleted_df.to_excel("ラーメン調査_仮想データ__MA処理済みデータ.xlsx", index=False)

    num  Q1_3_1_1：休日の学校運動部活動の段階的な地域移行について  Q1_3_1_1：その他内容  \
0     1                                 2             NaN   
1     2                                 2             NaN   
2     3                                 3             NaN   
3     4                                 2             NaN   
4     5                                 1             NaN   
..  ...                               ...             ...   
87   88                                 1             NaN   
88   89                                 2             NaN   
89   90                                 2             NaN   
90   91                                 2             NaN   
91   92                                 3             NaN   

    Q1_3_1_R3_2:休日の学校運動部活動の段階的な地域移行の実施主体を担うことについて  \
0                                               4   
1                                               5   
2                                               3   
3                                               4   
4 

In [41]:
# 3.SA・MA・FAを集計して、出力したい
## 3.1.データラベルのリストを読み込む
data_labels_df = pd.read_excel('/content/drive/MyDrive/ラーメン調査_仮想データ_データラベル.xlsx') # データラベルのファイル名orファイルのパスを入力
data_labels = {}
for column in data_labels_df.columns:
    data_labels[column] = data_labels_df[column].dropna().tolist()

## 3.2.SAの集計・出力
### 3.2.1.SAとして出力してほしい列名を入力
sa_columns = ['Q1_味付け', 'Q2_スープ']  # SAとして出力してほしい列名を入力
### 3.2.2.勝手に集計してくれるおまじない
for sa_column in sa_columns:
    # Check if the column exists in the DataFrame before accessing it
    if sa_column not in personal_information_deleted_df.columns:
        print(f"Warning: Column '{sa_column}' not found in DataFrame. Skipping.")
        continue  # Skip to the next column if not found

    sa_counts = personal_information_deleted_df[sa_column].value_counts().sort_index()
    sa_output_df = pd.DataFrame(sa_counts).reset_index()
    sa_output_df.columns = ['Value', 'Count']
    sa_output_df['Data Label'] = sa_output_df['Value'].apply(lambda x: data_labels.get(sa_column, [])[int(x) - 1] if pd.notna(x) else np.nan)
    # 勝手にパーセントの計算をしてくれるおまじない
    total_count = sa_output_df['Count'].sum()
    sa_output_df['percent'] = sa_output_df['Count'] / total_count
    sa_output_df['%'] = sa_output_df['percent'] * 100
    # 勝手に日本語訳してくれるおまじない
    sa_output_df = sa_output_df[['Value', 'Data Label', 'Count', 'percent', '%']]
    sa_output_df = sa_output_df.rename(columns={
    'Value': '回答',
    'Data Label': 'データラベル',
    'Count': '回答数',
    'percent': 'パーセント',
    '%': '％'
    })
    # 勝手に集計結果を出力してくれるおまじない
    output_file_name = f"{sa_column}_output.xlsx"
    sa_output_df.to_excel(output_file_name, index=False)


In [42]:
# 3.3.MAの集計・出力
## 指定したい列名リスト（ここに必要な列名を追加）
specified_columns = ['Q3_トッピング']  # この段階で集計したいMAの分割前の列名を指定

ma_columns_all = []
for original_column, split_columns in column_mapping.items():
    if original_column in specified_columns:  # 指定した列のみ処理
        ma_columns_all.extend(split_columns)

ma_counts = personal_information_deleted_df[ma_columns_all].sum()
ma_output_df = pd.DataFrame(ma_counts).reset_index()
ma_output_df.columns = ['Column Name', 'Count']
ma_output_df['Data Label'] = ma_output_df['Column Name'].apply(
    lambda x: data_labels.get(x.rsplit('_', 1)[0], {})[int(x.rsplit('_', 1)[1]) -1 ]   if x.rsplit('_', 1)[0] in data_labels and x.rsplit('_', 1)[1].isdigit() else x
)

# 'invalid' と 'NA' が無効回答とNA回答を表す値とする
invalid_na_count = personal_information_deleted_df[ma_columns_all][personal_information_deleted_df[ma_columns_all].isin(['invalid', 'NA'])].sum().sum() # Sum of invalid and NA across all MA columns
total_valid_count = personal_information_deleted_df[ma_columns_all].sum().sum() - invalid_na_count  # Total count minus invalid and NA count

# Applying the fix to ensure a single value is returned by the lambda function
# Modified to handle potential KeyError and ensure a single value is returned
ma_output_df['percent'] = ma_output_df.apply(lambda row: row['Count'] / personal_information_deleted_df[row['Column Name'].rsplit('_', 1)[0]].notna().sum() if row['Column Name'].rsplit('_', 1)[0] in personal_information_deleted_df.columns and personal_information_deleted_df[row['Column Name'].rsplit('_', 1)[0]].notna().sum() != 0 else 0, axis=1)
#Adding check for 0 and returning 0 to avoid division by zero error
ma_output_df['%'] = ma_output_df['percent'] * 100

# 名前の修正
ma_output_df = ma_output_df[['Column Name', 'Data Label', 'Count', 'percent', '%']]
ma_output_df = ma_output_df.rename(columns={
    'Column Name': '列名',
    'Data Label': 'データラベル',
    'Count': '回答数',
    'percent': 'パーセント',
    '%': '％'
})
ma_output_df.to_excel(f'ma_output_{original_column}.xlsx', index=False)
print(f"{ma_output_df}")
print(f"Output for {original_column} saved to ma_output_{original_column}.xlsx")

ValueError: Cannot set a DataFrame with multiple columns to the single column percent

In [ ]:
# 3.4.FAの集計・出力
# 自由記述の列名を指定
fa_columns = ['Q2_スープ_その他', 'Q3_トッピング_その他']  # Changed to a list of strings

for column in fa_columns:
    fa_responses_df = personal_information_deleted_df[[column]].dropna(how='all') # or use .fillna('') as explained above
    fa_responses_df.to_excel(f'{column}_自由記述回答.xlsx', index=False)
    print(f'{column}の回答を「{column}_自由記述回答.xlsx」に出力しました。')

In [ ]:
# 3.5.特殊な処理が必要な項目を抜き出し専用のdfを作成
# 3.5.1.会員数関連の処理(このテスト用コードでは来店者数の仮カラム)
# 3.5.1.1.会員数を区分に分けて集計
# 3.5.1.1.1.年齢層と性別ごとの来店者数を集計
age_gender_counts = {
    "4～12歳": {
        "男性": personal_information_deleted_df['Q11_1_来店者数_4～12歳_男性'].sum(),
        "女性": personal_information_deleted_df['Q11_6_来店者数_4～12歳_女性'].sum(),
        "性別不明": personal_information_deleted_df['Q11_11_来店者数_4～12歳_性別不明'].sum()
    },
    "13～19歳": {
        "男性": personal_information_deleted_df['Q11_2_来店者数_13～19歳_男性'].sum(),
        "女性": personal_information_deleted_df['Q11_7_来店者数_13～19歳_女性'].sum(),
        "性別不明": personal_information_deleted_df['Q11_12_来店者数_13～19歳_性別不明'].sum()
    },
    "20～34歳": {
        "男性": personal_information_deleted_df['Q11_3_来店者数_20～34歳_男性'].sum(),
        "女性": personal_information_deleted_df['Q11_8_来店者数_20～34歳_女性'].sum(),
        "性別不明": personal_information_deleted_df['Q11_13_来店者数_20～34歳_性別不明'].sum()
    },
    "35～49歳": {
        "男性": personal_information_deleted_df['Q11_4_来店者数_35～49歳_男性'].sum(),
        "女性": personal_information_deleted_df['Q11_9_来店者数_35～49歳_女性'].sum(),
        "性別不明": personal_information_deleted_df['Q11_14_来店者数_35～49歳_性別不明'].sum()
    },
    "50歳以上": {
        "男性": personal_information_deleted_df['Q11_5_来店者数_50歳以上_男性'].sum(),
        "女性": personal_information_deleted_df['Q11_10_来店者数_50歳以上_女性'].sum(),
        "性別不明": personal_information_deleted_df['Q11_15_来店者数_50歳以上_性別不明'].sum()
    }
}

# データフレームを作成
number_of_member_df = pd.DataFrame(age_gender_counts)

# 合計列を追加
number_of_member_df['合計'] = number_of_member_df.sum(axis=1)

# 総数行を追加
number_of_member_df.loc['総数'] = number_of_member_df.sum()

# 割合を算出
def ratio_calculation_for_number_of_member(df):
  # 割合の列を先に追加
  for column in df.columns:
    if column != '合計':
      df[column + '_割合'] = df[column] / df['合計']

  # パーセントの列を後に追加
  for column in df.columns:
    if column != '合計' and not column.endswith('_割合'):  # 割合列は除外
      df[column + '_パーセント'] = df[column] / df['合計'] * 100
  # 列の並び順を調整
  ratio_columns = [col for col in df.columns if col.endswith('_割合')]
  percent_columns = [col for col in df.columns if col.endswith('_パーセント')]
  other_columns = [col for col in df.columns if not (col.endswith('_割合') or col.endswith('_パーセント'))]
  df = df[other_columns + ratio_columns + percent_columns]
  return df

# 関数を適用
number_of_member_df = ratio_calculation_for_number_of_member(number_of_member_df)
print(number_of_member_df)

# Excelファイルに出力
number_of_member_df.to_excel('会員数.xlsx')
print("会員数.xlsxに保存しました。")

# クラブによる総数の回答との齟齬をチェックしたいが…
# 一旦保留

# 3.5.1.2.クラブ会員数のカテゴリー変数を作成(このテスト用コードでは来店者数の仮カラム)
# 3.5.1.2.1.算出もとになるカラムを指定
sum_for_number_of_members_columns = personal_information_deleted_df[['Q11_1_来店者数_4～12歳_男性', 'Q11_6_来店者数_4～12歳_女性', 'Q11_11_来店者数_4～12歳_性別不明',
                                                                    'Q11_2_来店者数_13～19歳_男性', 'Q11_7_来店者数_13～19歳_女性', 'Q11_12_来店者数_13～19歳_性別不明',
                                                                    'Q11_3_来店者数_20～34歳_男性', 'Q11_8_来店者数_20～34歳_女性', 'Q11_13_来店者数_20～34歳_性別不明',
                                                                    'Q11_4_来店者数_35～49歳_男性','Q11_9_来店者数_35～49歳_女性', 'Q11_14_来店者数_35～49歳_性別不明',
                                                                    'Q11_5_来店者数_50歳以上_男性', 'Q11_10_来店者数_50歳以上_女性', 'Q11_15_来店者数_50歳以上_性別不明']]
personal_information_deleted_df['会員数_sum'] = sum_for_number_of_members_columns.sum(axis=1)
# 3.5.1.2.2.会員数のカテゴリー変数を作成する関数(1以上100以下、101以上300以下、301以上1000以下、1001以上の四パターンに処理)
def categorize_members(value):
    if value <= 1:
        return '1人以下'
    elif 1 < value <= 100:
        return '1人以上100人以下'
    elif 101 <= value <= 300:
        return '101人以上300人以下'
    elif 301 <= value <= 1000:
        return '301人以上1000人以下'
    else:
        return '1001人以上'
# 3.5.1.2.3.クラブ会員数のカテゴリー変数を作成
personal_information_deleted_df['会員数_cat'] = personal_information_deleted_df['会員数_sum'].apply(categorize_members)
print(personal_information_deleted_df[['会員数_sum', '会員数_cat']])
# 3.5.1.2.4.クラブ会員数をカテゴリー変数で集計
members_cat_counts = personal_information_deleted_df['会員数_cat'].value_counts().sort_index()
members_cat_df = pd.DataFrame(members_cat_counts).reset_index()
members_cat_df.columns = ['カテゴリー', '回答数']
members_cat_df['割合'] = members_cat_df['回答数'] / members_cat_df['回答数'].sum()
members_cat_df['パーセント'] = members_cat_df['回答数'] / members_cat_df['回答数'].sum() * 100
print(members_cat_df)
members_cat_df.to_excel('会員数_カテゴリー別.xlsx', index=False)
print("会員数_カテゴリー別.xlsxに保存しました。")

print('会員数関連の処理が終了しました。')

In [ ]:
# 3.5.特殊な処理が必要な項目を抜き出し専用のdfを作成
# 3.5.4.スタッフ
staff_df = personal_information_deleted_df[['Q5_スタッフ数','Q6_スタッフ給与平均額']]
staff_df['クラブごと給与支払総額'] = staff_df['Q5_スタッフ数'] * staff_df['Q6_スタッフ給与平均額']
# staff_calculation_dfを辞書として初期化
staff_calculation_df = {}
# 計算結果を辞書に格納
staff_calculation_df['都内クラブ総スタッフ数'] = staff_df['Q5_スタッフ数'].sum()
staff_calculation_df['都内クラブ総給与支払額'] = staff_df['クラブごと給与支払総額'].sum()
staff_calculation_df['都内クラブ平均給与支払額'] = staff_df['クラブごと給与支払総額'].sum() / staff_df['Q5_スタッフ数'].sum()
# 辞書をDataFrameに変換
staff_calculation_df = pd.DataFrame([staff_calculation_df])
staff_df.to_excel('スタッフ.xlsx', index=False)
print("スタッフ.xlsxに保存しました。")
staff_calculation_df.to_excel('スタッフ_計算結果.xlsx', index=False)
print("スタッフ_計算結果.xlsxに保存しました。")
print('スタッフ関連の処理が終了しました。')

In [ ]:
# 3.5.特殊な処理が必要な項目を抜き出し専用のdfを作成
# 3.5.4.正社員
employee_df = personal_information_deleted_df[['Q7_正社員数', 'Q8_正社員給与平均額', 'Q9_正社員のうち暖簾分け希望者数', 'Q10 _正社員のうち暖簾分け希望者の給与平均額']]
employee_df['クラブごと正社員への給与支払総額'] = employee_df['Q7_正社員数'] * employee_df['Q8_正社員給与平均額']
employee_df['クラブごと暖簾分け希望正社員への給与支払総額'] = employee_df['Q9_正社員のうち暖簾分け希望者数'] * employee_df['Q10 _正社員のうち暖簾分け希望者の給与平均額']
# employee_calculation_dfを辞書として初期化
employee_calculation_df = {}
# 計算結果を辞書に格納
employee_calculation_df['都内クラブ総正社員数'] = employee_df['Q7_正社員数'].sum()
employee_calculation_df['都内クラブ総給与支払額'] = employee_df['クラブごと正社員への給与支払総額'].sum()
employee_calculation_df['都内クラブ暖簾分け希望正社員総数'] = employee_df['Q9_正社員のうち暖簾分け希望者数'].sum()
employee_calculation_df['都内クラブ平均給与支払額'] = employee_df['クラブごと正社員への給与支払総額'].sum() / employee_df['Q7_正社員数'].sum()
employee_calculation_df['都内クラブ暖簾分け希望正社員給与支払平均額'] = employee_df['クラブごと暖簾分け希望正社員への給与支払総額'].sum() / employee_df['Q9_正社員のうち暖簾分け希望者数'].sum()
# 辞書をDataFrameに変換
employee_calculation_df = pd.DataFrame([employee_calculation_df])
employee_df.to_excel('正社員.xlsx', index=False)
print("正社員.xlsxに保存しました。")
employee_calculation_df.to_excel('正社員_計算結果.xlsx', index=False)
print("正社員_計算結果.xlsxに保存しました。")
print('正社員関連の処理が終了しました。')

In [ ]:
# 3.5.特殊な処理が必要な項目を抜き出し専用のdfを作成
# 3.5.5.予算
budget_df = personal_information_deleted_df[['Q4_1_売上額', 'Q4_2_仕入額','Q4_3_サービス費', 'Q4_4_水道光熱費', 'Q4_5_家賃', 'Q4_6_消耗品費', 'Q4_7_人件費']]
budget_df['予算総額'] = budget_df[['Q4_2_仕入額', 'Q4_3_サービス費', 'Q4_4_水道光熱費', 'Q4_5_家賃', 'Q4_6_消耗品費', 'Q4_7_人件費']].sum(axis=1) #この計算式は実は正しくない（複数項目を合計する挙動を確かめる用）
def categorize_budget(value):
    if value <= 1:
        return '1円以下'
    elif 1 < value <= 10000000:
        return '1～10,000,000円以下'
    elif 10000001 <= value <= 20000000:
        return '10,000,001～20,000,000円以下'
    elif 20000001 <= value <= 30000000:
        return '20,000,001～30,000,000円以下'
    elif 30000001 <= value <= 40000000:
        return '30,000,001～40,000,000円以下'
    elif 40000001 <= value <= 50000000:
        return '40,000,001～50,000,000円以下'
    elif 50000001 <= value <= 60000000:
        return '50,000,001～60,000,000円以下'
    elif 60000001 <= value <= 70000000:
        return '60,000,001～70,000,000円以下'
    elif 70000001 <= value <= 80000000:
        return '70,000,001～80,000,000円以下'
    elif 80000001 <= value <= 90000000:
        return '80,000,001～90,000,000円以下'
    elif 90000001 <= value <= 100000000:
        return '90,000,001～100,000,000円以下'
    else:
        return '100,000,000円以上'
budget_df['予算総額_cat'] = budget_df['予算総額'].apply(categorize_budget)

budget_df['自己財源率'] = budget_df['Q4_1_売上額'] / budget_df['予算総額']
def categorize_own_financial_resources_ratio(value):
    if value <= 0.1:
        return '0～10%'
    elif 0.1 < value <= 0.3:
        return '11～30%'
    elif 0.3 < value <= 0.5:
        return '31～50%'
    elif 0.5 < value <= 0.7:
        return '51～70%'
    elif 0.7 < value <= 0.9:
        return '71～90%'
    else:
        return '91～100%'
budget_df['自己財源率_cat'] = budget_df['自己財源率'].apply(categorize_own_financial_resources_ratio)
budget_size_counts = budget_df['予算総額_cat'].value_counts().sort_index()
budget_size_df = pd.DataFrame(budget_size_counts).reset_index()
budget_size_df.columns = ['カテゴリー', '回答数']
budget_size_df['割合'] = budget_size_df['回答数'] / budget_size_df['回答数'].sum()
budget_size_df['パーセント'] = budget_size_df['回答数'] / budget_size_df['回答数'].sum() * 100
print(budget_size_df)
budget_size_df.to_excel('予算_カテゴリー別.xlsx', index=False)

own_financial_resources_ratio_counts = budget_df['自己財源率_cat'].value_counts().sort_index()
own_financial_resources_ratio_df = pd.DataFrame(own_financial_resources_ratio_counts).reset_index()
own_financial_resources_ratio_df.columns = ['カテゴリー', '回答数']
own_financial_resources_ratio_df['割合'] = own_financial_resources_ratio_df['回答数'] / own_financial_resources_ratio_df['回答数'].sum()
own_financial_resources_ratio_df['パーセント'] = own_financial_resources_ratio_df['回答数'] / own_financial_resources_ratio_df['回答数'].sum() * 100
print(own_financial_resources_ratio_df)
own_financial_resources_ratio_df.to_excel('自己財源率_カテゴリー別.xlsx', index=False)
print('予算関連の処理が終了しました。')